<a href="https://colab.research.google.com/github/SuphanutN/geo_spellchecker/blob/master/Spell_Checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas

In [ ]:
pip install PyDrive

In [ ]:
!pip install pythainlp

     |████████████████████████████████| 13.1MB 319kB/s 
     |████████████████████████████████| 747kB 44.3MB/s 


In [ ]:
import pandas as pd

import pythainlp

from pythainlp.corpus.common import thai_words
from pythainlp.tokenize import word_tokenize, DEFAULT_WORD_DICT_TRIE
from pythainlp.util import trie

from pythainlp import spell
from pythainlp.spell import NorvigSpellChecker

import pickle

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':"1rW9WWcYz0025dLHGFsh70K5QJ0SwsTB4"}) 
downloaded.GetContentFile('geocode.csv')
downloaded2 = drive.CreateFile({'id':"1d5NDfuaBlCAyEzMWhcvpjMSmIpr2Sz8y"}) 
downloaded2.GetContentFile('longdomap-poi-contributed-2020070921.csv')

In [ ]:
geo_file = pd.read_csv('geocode.csv')
map_file = pd.read_csv('longdomap-poi-contributed-2020070921.csv')

In [ ]:
geo_file.head()

,ProvinceCode,ProvinceTH,ProvinceEN,AmporCode,AmporTH,AmporEN,TumbolCode,TumbolTH,TumbolEN
0,81,กระบี่,Krabi,8103,เกาะลันตา,Ko Lanta,810303,เกาะกลาง,Ko Klang
1,81,กระบี่,Krabi,8103,เกาะลันตา,Ko Lanta,810302,เกาะลันตาน้อย,Ko Lanta Noi
2,81,กระบี่,Krabi,8103,เกาะลันตา,Ko Lanta,810301,เกาะลันตาใหญ่,Ko Lanta Yai
3,81,กระบี่,Krabi,8103,เกาะลันตา,Ko Lanta,810304,คลองยาง,Khlong Yang
4,81,กระบี่,Krabi,8103,เกาะลันตา,Ko Lanta,810305,ศาลาด่าน,Sala Dan


In [ ]:
map_file.head()

,id,name_th,name_en,address_th,address_en,latitude,longitude,telephone,website,lastupdate,poi_type,username
0,10000001,บริษัท เมตามีเดีย เทคโนโลยี จำกัด,"Metamedia Technology Co., Ltd.",98 อาคารสาทร สแควร์ ออฟฟิศ ทาวเวอร์ ชั้น 4 ห้อ...,"Rm. 407, 4F, Sathorn Square Office Tower, 98 N...",13.722642,100.529316,+6621081790,http://www.mm.co.th,2020-04-17 02:12:58,building,pattara
1,10000003,ไทยอินโฟนิตี้,Thai Infonity,แขวงทุ่งสองห้อง เขตหลักสี่ กรุงเทพมหานคร 10210,"Thung Song Hong, Lak Si, Bangkok, 10210",13.887970,100.575757,NaN,NaN,2020-01-08 22:28:11,NaN,pattara
2,10000007,ตลาดหลักทรัพย์แห่งประเทศไทย,The Stock Exchange of Thailand,ถนนรัชดาภิเษก แขวงคลองเตย เขตคลองเตย กรุงเทพม...,NaN,13.722620,100.559362,0-2229-2000 หรือ 0-2229-2222,http://www.set.or.th,2020-01-08 22:28:11,Shop / Other public location,pattara
3,10000008,ธนาบุตร,ธนาบุตร,บริษัท ธนาบุตร จำกัด,"Rama 1 Rd., Wang Mai, Patumwan, Bangkok, 10400",13.746797,100.528862,NaN,http://www.tanabutr.co.th/,2020-01-08 22:28:11,building,pattara
4,10000009,คลินิกแพทย์หญิงเฟื่องเพชร,คลินิกแพทย์หญิงเฟื่องเพชร,486/130 ซอยเพชรบุรี 16 แขวงถนนเพชรบุรี เขตราชเ...,"486/130 Soi Phetburi 16, Thanon Phet Buri, Rat...",13.752618,100.531037,+6622156159,NaN,2020-01-08 22:28:11,"hospital, clinic, healthcare office",pattara


In [ ]:
province_list_dup = geo_file['ProvinceTH'].tolist()
ampor_list_dup = geo_file['AmporTH'].tolist()
tumbol_list_dup = geo_file['TumbolTH'].tolist()
all_word_dup = province_list_dup + ampor_list_dup + tumbol_list_dup

word_freq = {}
for word in all_word_dup:
  if word not in word_freq:
      word_freq[word] = 0
  word_freq[word] += 1

In [ ]:
province_list = geo_file['ProvinceTH'].unique().tolist()
ampor_list = geo_file['AmporTH'].unique().tolist()
tumbol_list = geo_file['TumbolTH'].unique().tolist()
custom_list = ["เจริญกรุง"]
all_word = province_list + ampor_list + tumbol_list + custom_list

custom_dict_geo = set(thai_words())
for word in all_word:
  custom_dict_geo.add(word)

custom_dict_trie = trie.dict_trie(dict_source=custom_dict_geo)

test_text = "ถนนพุทธมณฑลสาย 4 ศาลายา พุทธมณฑล จ.นครปฐม 73170"

geo_stopword = ["ถนน","ตำบล","แขวง","อำเภอ","เขต","จังหวัด","ถ.","ต.","บ.","จ.","ประเทศ"]

token_test_text = word_tokenize(test_text, engine="newmm", custom_dict=custom_dict_trie, keep_whitespace=False)
for word in geo_stopword:
  if word in token_test_text:
    token_test_text.remove(word)

print(token_test_text)

['พุทธมณฑล', 'สาย', '4', 'ศาลายา', 'พุทธมณฑล', 'นครปฐม', '73170']


In [ ]:
address_list = map_file['address_th'].tolist()

for doc in address_list:
  token_doc = word_tokenize(doc, engine="newmm", custom_dict=custom_dict_trie, keep_whitespace=False)
  for word in geo_stopword:
    if word in token_doc:
      token_doc.remove(word)
      
  for word in token_doc:
    if word not in word_freq:
        word_freq[word] = 0
    word_freq[word] += 1


In [ ]:
geo_dictlist = []
for key, value in word_freq.items():
    temp = (key,value)
    geo_dictlist.append(temp)

In [ ]:
print(geo_dictlist[:20])
print(word_freq["กรุงเทพ"])

[('กระบี่', 4075), ('กรุงเทพมหานคร', 71880), ('กาญจนบุรี', 5273), ('กาฬสินธุ์', 5899), ('กำแพงเพชร', 4658), ('ขอนแก่น', 13727), ('จันทบุรี', 4068), ('ฉะเชิงเทรา', 5410), ('ชลบุรี', 17181), ('ชัยนาท', 2759), ('ชัยภูมิ', 6834), ('ชุมพร', 3589), ('เชียงราย', 8377), ('เชียงใหม่', 17390), ('ตรัง', 4042), ('ตราด', 2985), ('ตาก', 3599), ('นครนายก', 2327), ('นครปฐม', 9715), ('นครพนม', 4293)]
770


In [ ]:
geo_spell_checker = NorvigSpellChecker(custom_dict=geo_dictlist,min_freq=5)
test_text = "บางกรวย"
allword_prob = geo_spell_checker.spell(test_text)
prob = geo_spell_checker.prob(test_text)
predict_word = allword_prob[0]

print(allword_prob)
print(prob)
print(predict_word)

['บางกรวย']
0.0008694904367467588
บางกรวย


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!ls '/content/gdrive/My Drive/Colab Notebooks/raw_file/spell_checker'

dict.csv  dict.pkl  geocode.csv  longdomap-poi-contributed-2020070921.csv


In [ ]:
save_path = F"/content/gdrive/My Drive/Colab Notebooks/raw_file/spell_checker/" 

dict_file = open(save_path+"dict.pkl", "wb")
pickle.dump(geo_dictlist, dict_file)